In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
#submission file 읽기
df = pd.read_csv('/opt/ml/output.csv')

data = dict()
data['info'] = {
    "year": 2021,
    "version": "1.0",
    "description": "Recycle Trash",
    "contributor": "Upstage",
    "url": 0,
    "date_created": "2021-02-02 01:10:00"
  }

data['license'] = {
      "id": 0,
      "name": "CC BY 4.0",
      "url": "https://creativecommons.org/licenses/by/4.0/deed.ast"
    }

data['categories'] = [
    {
      "id": 0,
      "name": "General trash",
      "supercategory": "General trash"
    },
    {
      "id": 1,
      "name": "Paper",
      "supercategory": "Paper"
    },
    {
      "id": 2,
      "name": "Paper pack",
      "supercategory": "Paper pack"
    },
    {
      "id": 3,
      "name": "Metal",
      "supercategory": "Metal"
    },
    {
      "id": 4,
      "name": "Glass",
      "supercategory": "Glass"
    },
    {
      "id": 5,
      "name": "Plastic",
      "supercategory": "Plastic"
    },
    {
      "id": 6,
      "name": "Styrofoam",
      "supercategory": "Styrofoam"
    },
    {
      "id": 7,
      "name": "Plastic bag",
      "supercategory": "Plastic bag"
    },
    {
      "id": 8,
      "name": "Battery",
      "supercategory": "Battery"
    },
    {
      "id": 9,
      "name": "Clothing",
      "supercategory": "Clothing"
    }
  ]

pseudo_annotations = []
pseudo_images = []
cnt = 0
for i in df.index:
    pred= df['PredictionString'][i]
    bbox = pred.split()
    bbox = map(float, bbox)
    bbox = list(bbox)
    bbox = np.array(bbox).reshape(-1,6)
    
    
    pseudo_images.append(
                {
                    "width":1024,
                    "height":1024,
                    "file_name":df['image_id'][i],
                    "license":0,
                    "flickr_url":0,
                    "coco_url":0,
                    "date_captured":0,
                    "id":i
                }
            )
    
    for d in bbox:
        if d[1] > 0.6: # 성능: 0.75 이상
            x_min, y_min, x_max, y_max = tuple(d[2:])
            w, h = x_max - x_min, y_max - y_min
            
            pseudo_annotations.append(
                {
                    "image_id": i,
                    "area": w * h,
                    "category_id": int(d[0]),
                    "bbox": [x_min, y_min, w, h],
                    "iscrowd": 0,
                    "id": cnt
                }
            )

            cnt +=1
    
data["images"] = pseudo_images
data["annotations"] = pseudo_annotations

print(data)
#CoCo dataset 파일로 저장
with open('pseudo.json', 'w', encoding="utf-8") as make_file:
    json.dump(data, make_file, ensure_ascii=False, indent="\t")

{'info': {'year': 2021, 'version': '1.0', 'description': 'Recycle Trash', 'contributor': 'Upstage', 'url': 0, 'date_created': '2021-02-02 01:10:00'}, 'license': {'id': 0, 'name': 'CC BY 4.0', 'url': 'https://creativecommons.org/licenses/by/4.0/deed.ast'}, 'categories': [{'id': 0, 'name': 'General trash', 'supercategory': 'General trash'}, {'id': 1, 'name': 'Paper', 'supercategory': 'Paper'}, {'id': 2, 'name': 'Paper pack', 'supercategory': 'Paper pack'}, {'id': 3, 'name': 'Metal', 'supercategory': 'Metal'}, {'id': 4, 'name': 'Glass', 'supercategory': 'Glass'}, {'id': 5, 'name': 'Plastic', 'supercategory': 'Plastic'}, {'id': 6, 'name': 'Styrofoam', 'supercategory': 'Styrofoam'}, {'id': 7, 'name': 'Plastic bag', 'supercategory': 'Plastic bag'}, {'id': 8, 'name': 'Battery', 'supercategory': 'Battery'}, {'id': 9, 'name': 'Clothing', 'supercategory': 'Clothing'}], 'images': [{'width': 1024, 'height': 1024, 'file_name': 'test/0000.jpg', 'license': 0, 'flickr_url': 0, 'coco_url': 0, 'date_cap